***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [23]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  PRIMARY_WORKER_COUNT  SECONDARY_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-a697  GCE       3                                             RUNNING  us-central1-a


# Imports & Setup

In [24]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [25]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage
import math
import builtins as pybuiltins


import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [26]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Feb 28 11:22 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [27]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [28]:
spark

In [29]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = '207219783b2' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh' and not b.name.startswith("postings_gcp"):
        paths.append(full_path+b.name)

In [30]:
parquetFile = spark.read.parquet(*paths)
doc_title_pairs = parquetFile.select("title", "id").rdd

In [31]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [32]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

24/02/28 13:12:36 WARN SparkContext: The path /home/dataproc/inverted_index_gcp.py has been added already. Overwriting of added paths is not supported in the current version.


In [33]:
from inverted_index_gcp import InvertedIndex

**Project**


In [34]:
# stop words
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ['category', 'references', 'also', 'links', 'extenal', 'see', 'thumb']
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

all_stopwords = english_stopwords.union(corpus_stopwords)


# tokenize and stemming
def tokenize_stemming(text):
    list_token = []
    stemmer = PorterStemmer()
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    for token in tokens:
        if token not in english_stopwords:
            list_token.append(stemmer.stem(token))
    return list_token


In [35]:
# tf
# normalizion by max frequency term
def term_frequency(text, id):
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]

    # Count the frequency of each word that is not a stopword
    word_freq = {}
    max_freq = 0
    # print(tokens)
    for token in tokens:
        if token not in all_stopwords:
            if token not in word_freq:
                word_freq[token] = 1
            else:
                word_freq[token] += 1

    lst_tuples = [(token, (id, freq)) for token, freq in word_freq.items()]

    return lst_tuples


def reduce_word_counts(unsorted_pl):
  sorted_pl = sorted(unsorted_pl, key=lambda x: x[0])
  return sorted_pl

# df
def calculate_df(postings):
  rdd_token_doc_freq = postings.map(lambda x: (x[0], len(x[1]))).reduceByKey(lambda a, b: a + b)
  return rdd_token_doc_freq


# doc len
def doc_lenght(text, doc_id):
  tokens = tokenize_stemming(text)
  return((doc_id,len(tokens)))


In [36]:


NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def partition_postings_and_write(postings, index_name):
  def map_to_buckets(pair):
        word, posting_list = pair
        bucket_id = token2bucket_id(word)
        return bucket_id, [(word, posting_list)]

  def reduce_to_disk(bucket, word_posting_lists):
    # Write posting lists for the bucket to disk
    posting_locs = InvertedIndex.write_a_posting_list((bucket, word_posting_lists), 'postings_gcp', bucket_name)
    return posting_locs

  # Partition the postings into buckets
  buckets = postings.map(map_to_buckets)

  # Group postings by bucket ID
  grouped_buckets = buckets.reduceByKey(lambda x, y: x + y)

  # Write each bucket to disk and get posting locations
  bucket_posting_locs = grouped_buckets.map(lambda x: reduce_to_disk(*x))

  return bucket_posting_locs

def create_posting_locs(index_name, postings_filtered):
  posting_locs_list = partition_postings_and_write(postings_filtered, index_name).collect()

  super_posting_locs = defaultdict(list)
  for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
    if not blob.name.endswith("pickle"):
        continue
    with blob.open("rb") as f:
        posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
            super_posting_locs[k].extend(v)
  return super_posting_locs

In [37]:
def create_index(index_name, doc_pairs):

    word_counts = doc_pairs.flatMap(lambda x: term_frequency(x[0], x[1]))
    postings = word_counts.groupByKey().mapValues(reduce_word_counts)
    postings_filtered = postings.filter(lambda x: len(x[1])>50)

    dl_rdd = doc_pairs.map(lambda x: doc_lenght(x[0], x[1]))
    dl = dl_rdd.collectAsMap()
    w2df = calculate_df(postings_filtered)
    w2df_dict = w2df.collectAsMap()

    # Create inverted index instance
    inverted = InvertedIndex()
    inverted.df = w2df_dict
    inverted.N = doc_pairs.count()
    inverted.doc_len = dl
    inverted.posting_locs = create_posting_locs(index_name, postings_filtered)

    inverted.write_index('.', f'index_{index_name}')

    index_src = f"index_{index_name}.pkl"
    index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
    !gsutil cp $index_src $index_dst
    
#     return inverted

In [38]:
TUPLE_SIZE = 6
TF_MASK = 2 ** 16 - 1 # Masking the 16 low bits of an integer
from contextlib import closing


def read_posting_list(inverted, w):
  with closing(MultiFileReader(".", bucket_name)) as reader:
    locs = inverted.posting_locs[w]
    b = reader.read(locs, inverted.df[w] * TUPLE_SIZE)
    posting_list = []
    for i in range(inverted.df[w]):
      doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
      tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
      posting_list.append((doc_id, tf))
    return posting_list

In [39]:
# weight = tf*idf
def calculate_tf_idf(index, term, tf, doc_id):
  # doc_len =  doc_lenght(x[0], x[1])
  tf_idf = (tf/index.doc_len[doc_id])* math.log2(index.N/index.df[term])
  return tf_idf


def cosine_similarity(query, index):
    """ Returns: {doc_id:cosine score} """

    dict_cosine_sim = {}
    doc_weights_dict = {}
    query_dict = dict(term_frequency(query ,0))

    # create lists of keys and values from the query_dict
    query_list_keys = list(query_dict.keys())
    query_list_values = list(query_dict.values())

    # create list of items from index.df dictionery keys
    index_df_list_keys = list(index.df.keys())


    for term in query_list_keys:
      w_term_query = (query_dict[term][1]/len(query))* math.log2(index.N/index.df[term])

      if term in index_df_list_keys:
        posting_list = index.read_a_posting_list(".", term, bucket_name)
        for doc_id, freq in posting_list:
          w_term_doc = calculate_tf_idf(index, term, freq, doc_id)
          if doc_id in dict_cosine_sim.keys():
            dict_cosine_sim[doc_id] += (w_term_doc)*(w_term_query)
            doc_weights_dict[doc_id] += (w_term_doc) ** 2
          else:
            dict_cosine_sim[doc_id] = (w_term_doc)*(w_term_query)
            doc_weights_dict[doc_id] = (w_term_doc) ** 2


    for doc_id in list(dict_cosine_sim.keys()):
      Word_doc_id__weight = doc_weights_dict[doc_id];
      dict_cosine_sim[doc_id] /= ( math.sqrt(pybuiltins.sum(value[1] ** 2 for value in query_list_values) * Word_doc_id__weight ))



    sorted_docs = sorted(dict_cosine_sim.items(), key=lambda x: x[1], reverse=True)

    print(sorted_docs)
    return dict_cosine_sim


In [51]:
import time
def search_by_title(inverted, query):
    """ Returns: n best docs """
    start_time = time.time()  # Record the start time

    query_filterd = tokenize_stemming(query)
    cosSim_score = cosine_similarity(query, inverted)

    end_time = time.time()  # Record the end time
    execution_time = end_time - start_time  # Calculate the execution time
    print("Function execution time:", execution_time, "seconds")
    return cosSim_score

In [41]:
create_index("title", doc_title_pairs)
idx = InvertedIndex.read_index("postings_gcp", 'index_title', bucket_name)


Copying file://index_title.pkl [Content-Type=application/octet-stream]...
- [1 files][ 43.3 MiB/ 43.3 MiB]                                                
Operation completed over 1 objects/43.3 MiB.                                     


In [52]:
query = "genetic"
res = search_by_title(idx, query)

[(12383, 2.1214120923781348), (12385, 2.1214120923781348), (12424, 2.1214120923781348), (12437, 2.1214120923781348), (12942, 2.1214120923781348), (40254, 2.1214120923781348), (44353, 2.1214120923781348), (59013, 2.1214120923781348), (72016, 2.1214120923781348), (144854, 2.1214120923781348), (157941, 2.1214120923781348), (158005, 2.1214120923781348), (251487, 2.1214120923781348), (274192, 2.1214120923781348), (310782, 2.1214120923781348), (340952, 2.1214120923781348), (347560, 2.1214120923781348), (364423, 2.1214120923781348), (371709, 2.1214120923781348), (403627, 2.1214120923781348), (550948, 2.1214120923781348), (554546, 2.1214120923781348), (555213, 2.1214120923781348), (555480, 2.1214120923781348), (562180, 2.1214120923781348), (887836, 2.1214120923781348), (901149, 2.1214120923781348), (1090609, 2.1214120923781348), (1105886, 2.1214120923781348), (1217294, 2.1214120923781348), (1325445, 2.1214120923781348), (1431111, 2.1214120923781348), (1505166, 2.1214120923781348), (1652585, 2.